## Part 1

In [ ]:
import requests

path = 'http://text.npr.org'
r = requests.get(path)

text_to_process = r.text

link_queue = set()
while text_to_process.find('href="') >= 0:
    text_to_process = text_to_process[text_to_process.find('href="') + 6:]
    link = text_to_process[:text_to_process.find('"')]
    ## Add http here in part 2
    
    link_queue.add(link)
    text_to_process = text_to_process[text_to_process.find('"') + 1:]

## Part 2

In [ ]:
import requests

path = 'http://text.npr.org'
r = requests.get(path)

text_to_process = r.text

link_queue = set()
while text_to_process.find('href="') >= 0:
    text_to_process = text_to_process[text_to_process.find('href="') + 6:]
    link = text_to_process[:text_to_process.find('"')]
    if not link.startswith('http'):
        link = path + link
    link_queue.add(link)
    text_to_process = text_to_process[text_to_process.find('"') + 1:]

## Part 3

In [ ]:
queue_to_return = set()

for link in link_queue:
    if 'npr.org' in link:
        if  'sid=' in link.lower() or 'tid=' in link.lower():
            queue_to_return.add(link)
        
queue_to_return

## Part 4

In [ ]:
def get_links(html_text):
    link_queue = set()
    while html_text.find('href="') >= 0:
        html_text = html_text[html_text.find('href="') + 6:]
        link = html_text[:html_text.find('"')]
        
        if not link.startswith('http'):
            link = path + link
        
        if 'npr.org' in link:
            if  'sid=' in link.lower() or 'tid=' in link.lower():
                link_queue.add(link)
        
        html_text = html_text[html_text.find('"') + 1:]
        
    return link_queue

link_queue = get_links(r.text)
link_queue

## Part 5

In [ ]:
article_html = requests.get("http://text.npr.org/s.php?sId=575392333").text
article_html

## Part 6

In [ ]:
paragraphs = []
while article_html.find('<p>') >= 0:
    article_html = article_html[article_html.find('<p>') + 3:]
    paragraph = article_html[:article_html.find('</p>')]
    
    print(paragraph)
    paragraphs.append(paragraph)
    
    article_html = article_html[article_html.find('</p>') + 4:]

In [ ]:
cleaned_paragraphs = []
for paragraph in paragraphs:
    new_paragraph = ""
    
    tag = False
    for char in paragraph:
        
        if char == "<":
            tag = True
            
        if not tag:
            new_paragraph += char
            
        if char == ">":
            tag = False
        
        
    if len(new_paragraph.strip()) > 0:
        cleaned_paragraphs.append(new_paragraph)

In [ ]:
for paragraph in cleaned_paragraphs:
    print(paragraph + '\n')

## Part 7

In [ ]:
def parse_text(article_html):
    paragraphs = []
    while article_html.find('<p>') >= 0:
        article_html = article_html[article_html.find('<p>') + 3:]
        paragraph = article_html[:article_html.find('</p>')]

        new_paragraph = ""
    
        tag = False
        for char in paragraph:

            if char == "<":
                tag = True
            if not tag:
                new_paragraph += char
            if char == ">":
                tag = False

        if len(new_paragraph.strip()) > 0:
            paragraphs.append(new_paragraph)

        article_html = article_html[article_html.find('</p>') + 4:]
        
    return "\n\n".join(paragraphs)

In [ ]:
article_html = requests.get("http://text.npr.org/s.php?sId=575392333").text

print(parse_text(article_html))

## Part 8

In [ ]:
def get_id(url):
    url = url[url.lower().find('sid=') + 4:]
    if '&' in url:
        url = url[:url.find('&')]
        
    return url

In [ ]:
import os

starting_url = 'http://text.npr.org'
folder_out   = 'data/scraper_test/'

if not os.path.exists(folder_out):
    os.makedirs(folder_out)

links_visited = set()
links_queue   = get_links(requests.get(starting_url).text)

while len(links_queue) > 0:
    
    link = links_queue.pop()
    links_visited.add(link)
    
    html = requests.get(link).text
    
    urls = get_links(html)
    for url in urls:
        if not url in links_visited:
            links_queue.add(url)
            
    if 'sid=' in link.lower():
        text = parse_text(html)
        article_id = get_id(link)
        
        with open(folder_out + article_id + ".txt", "w") as ft_hdl:
            ft_hdl.write(text)